## Попрактикуемся с тем, что изучили
Будем практиковаться на датасете: https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:

Создать Dataset для загрузки данных (используем только числовые данные)
Обернуть его в Dataloader
Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели
train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/PyTorch_framework_for_the_development_of_artificial_neural_networks/Data/train.csv')
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [6]:
data.activation_date = pd.to_datetime(data.activation_date)
data['day_of_month'] = data.activation_date.apply(lambda x: x.day)
data['day_of_week'] = data.activation_date.apply(lambda x: x.weekday())

In [7]:
agg_cols = ['region', 'category_name',
            'city', 'user_type', 'parent_category_name']
for col in tqdm_notebook(agg_cols):
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    gp = data.groupby(col)['deal_probability']
    mean = gp.mean()
    data[col + '_deal_probability_avg'] = data[col].map(mean)

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg,parent_category_name_deal_probability_avg
0,b912c3c6a6ad,e00f8ff2eaf9,19,460,4,42,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,1,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,28,1,0.122004,0.198445,0.123397,0.149557,0.075876
1,2dac0150717d,39aeb48f0017,17,1300,2,22,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,1,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,26,6,0.136721,0.191848,0.139400,0.149557,0.179633
2,ba83aefab5dc,91e2f88dd6e3,16,1276,0,2,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,1,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,20,0,0.135944,0.171572,0.124881,0.149557,0.175421
3,02996f1dd2ea,bf5cccea572d,21,940,4,42,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,0,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,25,5,0.142602,0.198445,0.135031,0.124513,0.075876
4,7c90be56d2ab,ef50846afc0b,4,317,6,0,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,1,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,16,3,0.145908,0.278427,0.137275,0.149557,0.263336


In [9]:
data = data.drop(['user_id', 'description', 'image', 'parent_category_name',
                    'item_id', 'param_1', 'param_2', 'param_3', 'title', 'activation_date'], axis=1)

In [10]:
for col in data.columns:
    if data[col].isna().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)

In [11]:
display(data.head(2), data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 15 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   region                                     1503424 non-null  int64  
 1   city                                       1503424 non-null  int64  
 2   category_name                              1503424 non-null  int64  
 3   price                                      1503424 non-null  float64
 4   item_seq_number                            1503424 non-null  int64  
 5   user_type                                  1503424 non-null  int64  
 6   image_top_1                                1503424 non-null  float64
 7   deal_probability                           1503424 non-null  float64
 8   day_of_month                               1503424 non-null  int64  
 9   day_of_week                                1503424 non-null  int64  

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg,parent_category_name_deal_probability_avg
0,19,460,42,400.0,2,1,1008.0,0.12789,28,1,0.122004,0.198445,0.123397,0.149557,0.075876
1,17,1300,22,3000.0,19,1,692.0,0.00000,26,6,0.136721,0.191848,0.139400,0.149557,0.179633


None

In [12]:
train, test = train_test_split(data, test_size=0.25, random_state=13)
train.head(2)

,region,city,category_name,price,item_seq_number,user_type,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg,parent_category_name_deal_probability_avg
31723,9,755,9,1300.0,6,1,1056.0,0.0,23,3,0.147066,0.178480,0.189069,0.149557,0.263336
929954,19,460,29,1300.0,16,1,399.0,0.0,28,1,0.122004,0.046447,0.123397,0.149557,0.075876


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127568 entries, 31723 to 1015882
Data columns (total 15 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   region                                     1127568 non-null  int64  
 1   city                                       1127568 non-null  int64  
 2   category_name                              1127568 non-null  int64  
 3   price                                      1127568 non-null  float64
 4   item_seq_number                            1127568 non-null  int64  
 5   user_type                                  1127568 non-null  int64  
 6   image_top_1                                1127568 non-null  float64
 7   deal_probability                           1127568 non-null  float64
 8   day_of_month                               1127568 non-null  int64  
 9   day_of_week                                1127568 non-null  int

In [14]:
class DFDataset(torch.utils.data.Dataset):
    def __init__(self, df, normalize=False, fit_scaler=False):
        self.df = df.copy()
        self.normalize = normalize
        self.scaler = MinMaxScaler()
        self.fit_scaler = fit_scaler
        self.sc_fl = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx, -1:]
        if self.fit_scaler:
            self.scaler.fit(df.iloc[:, :-1])
            self.sc_fl = 1

        if self.normalize and sc_fl:
            df = scaler.transform(df)

        tensor = torch.FloatTensor(self.df.iloc[idx, 1:-1].values)
        label = torch.FloatTensor(label.values)

        return tensor, label

In [15]:
train_dataset = DFDataset(train)
test_dataset = DFDataset(test)

In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2048,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=2048,
                                          shuffle=True,
                                          num_workers=4)

In [17]:
class FFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FFNet, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 5*hidden_dim)
        self.dp1 = nn.Dropout(0.40)

        self.bn2 = nn.BatchNorm1d(5*hidden_dim)
        self.fc2 = nn.Linear(5*hidden_dim, 2*hidden_dim)
        self.dp2 = nn.Dropout(0.15)

        self.bn4 = nn.BatchNorm1d(2*hidden_dim)
        self.fc4 = nn.Linear(2*hidden_dim, 1)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.dp1(x)

        x = self.bn2(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.dp2(x)

        x = self.bn4(x)
        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

In [18]:
def rmsle_loss(y_pred, y_true):
    loss = torch.sqrt(torch.mean((torch.log(y_pred+1)-torch.log(y_true+1))**2))
    return loss

In [19]:
EPOCHES = 1
LR = 0.001

In [20]:
def train_loop(tr_dataloader, ev_dataloader, model, optimizer, history):

    size = len(tr_dataloader.dataset)
    for batch, (X, y) in enumerate(tr_dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = rmsle_loss(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = batch * len(X)
            history['train'].append(loss)
            test_loss = eval_loop(ev_dataloader, model)
            history['eval'].append(test_loss)
            print(f"loss: {loss:>7f}, Avg test loss: {test_loss:>8f}  [{current:>5d}/{size:>5d}]")

    return history


def eval_loop(ev_dataloader, model):
    
    size = len(ev_dataloader.dataset)
    num_batches = len(ev_dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in ev_dataloader:
            pred = model(X.to(device))
            test_loss += rmsle_loss(pred, y.to(device))

    test_loss /= num_batches
    
    return test_loss

## SGD

In [21]:
model = FFNet(13, 5).to(device)

In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [23]:
history_1 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_1 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_1
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.316888, Avg test loss: 0.316757  [    0/1127568]
loss: 0.315606, Avg test loss: 0.315383  [102400/1127568]
loss: 0.313848, Avg test loss: 0.314023  [204800/1127568]
loss: 0.310679, Avg test loss: 0.312584  [307200/1127568]
loss: 0.312598, Avg test loss: 0.311210  [409600/1127568]
loss: 0.310850, Avg test loss: 0.309807  [512000/1127568]
loss: 0.308516, Avg test loss: 0.308387  [614400/1127568]
loss: 0.306575, Avg test loss: 0.306951  [716800/1127568]
loss: 0.303570, Avg test loss: 0.305586  [819200/1127568]
loss: 0.304691, Avg test loss: 0.304093  [921600/1127568]
loss: 0.303313, Avg test loss: 0.302717  [1024000/1127568]
loss: 0.302389, Avg test loss: 0.301261  [642400/1127568]
Done!


## ADAM

In [24]:
model = FFNet(13, 5).to(device)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [27]:
history_2 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_2 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_2
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.294905, Avg test loss: 0.295784  [    0/1127568]
loss: 0.267115, Avg test loss: 0.266969  [102400/1127568]
loss: 0.235162, Avg test loss: 0.233326  [204800/1127568]
loss: 0.181692, Avg test loss: 0.181391  [307200/1127568]
loss: 0.118856, Avg test loss: 0.119320  [409600/1127568]
loss: 0.070851, Avg test loss: 0.069099  [512000/1127568]
loss: 0.048351, Avg test loss: 0.048982  [614400/1127568]
loss: 0.044060, Avg test loss: 0.043328  [716800/1127568]
loss: 0.039648, Avg test loss: 0.039884  [819200/1127568]
loss: 0.036995, Avg test loss: 0.037387  [921600/1127568]
loss: 0.035028, Avg test loss: 0.035335  [1024000/1127568]
loss: 0.032600, Avg test loss: 0.033932  [642400/1127568]
Done!


## RMSProp

In [28]:
model = FFNet(13, 5).to(device)

In [29]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)

In [30]:
history_3 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_3 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_3
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.258845, Avg test loss: 0.248078  [    0/1127568]
loss: 0.164963, Avg test loss: 0.165204  [102400/1127568]
loss: 0.101467, Avg test loss: 0.100570  [204800/1127568]
loss: 0.062035, Avg test loss: 0.061349  [307200/1127568]
loss: 0.047176, Avg test loss: 0.047733  [409600/1127568]
loss: 0.043658, Avg test loss: 0.043589  [512000/1127568]
loss: 0.041090, Avg test loss: 0.040651  [614400/1127568]
loss: 0.039490, Avg test loss: 0.038427  [716800/1127568]
loss: 0.035163, Avg test loss: 0.036520  [819200/1127568]
loss: 0.034528, Avg test loss: 0.034902  [921600/1127568]
loss: 0.032864, Avg test loss: 0.033362  [1024000/1127568]
loss: 0.033125, Avg test loss: 0.032229  [642400/1127568]
Done!


## SGD + Momentum
Ради эксперимента попробуем применить SGD + Momentum

In [31]:
model = FFNet(13, 5).to(device)

In [32]:
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.8)

In [33]:
history_4 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_4 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_4
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.325045, Avg test loss: 0.323981  [    0/1127568]
loss: 0.316220, Avg test loss: 0.316779  [102400/1127568]
loss: 0.307952, Avg test loss: 0.308927  [204800/1127568]
loss: 0.299363, Avg test loss: 0.300936  [307200/1127568]
loss: 0.294981, Avg test loss: 0.292719  [409600/1127568]
loss: 0.284595, Avg test loss: 0.284382  [512000/1127568]
loss: 0.274855, Avg test loss: 0.276000  [614400/1127568]
loss: 0.268279, Avg test loss: 0.267472  [716800/1127568]
loss: 0.261819, Avg test loss: 0.258943  [819200/1127568]
loss: 0.249897, Avg test loss: 0.250346  [921600/1127568]
loss: 0.242463, Avg test loss: 0.241831  [1024000/1127568]
loss: 0.235039, Avg test loss: 0.233165  [642400/1127568]
Done!


Вывод: при одинаковой скорости обучения скорость схождения быстрее всего RMSProp, на втором месте - Adam. Худший результат у SGD. Сохранение градиентов в SGD + Momentum позволяет быстрее приийти к минимуму функции, нежели простой SGD.